In [1]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    # ('begin_inventory.csv', 'begin_inventory'),
    # ('end_inventory.csv', 'end_inventory'),
    # ('purchase_prices.csv', 'purchase_price'),
    # ('purchases.csv', 'purchases'),
    # ('sales.csv', 'sales'),
    # ('vendor_invoice.csv', 'vendor_invoice'),
    
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Asdf@1234',
    database='vendor'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'C:/Users/ary33/Desktop/Proj_data_analysis/Vendor/data'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing sales.csv
NaN values before replacement:
InventoryId       0
Store             0
Brand             0
Description       0
Size              0
SalesQuantity     0
SalesDollars      0
SalesPrice        0
SalesDate         0
Volume            0
Classification    0
ExciseTax         0
VendorNo          0
VendorName        0
dtype: int64

Processing vendor_invoice.csv
NaN values before replacement:
VendorNumber       0
VendorName         0
InvoiceDate        0
PONumber           0
PODate             0
PayDate            0
Quantity           0
Dollars            0
Freight            0
Approval        5169
dtype: int64



In [7]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer
from tqdm import tqdm
import time

# -------------------------------
# ⚙️ Database Configuration
# -------------------------------
username = "root"
password = "Asdf%401234"
host = "localhost"
database = "vendor"

# Create SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")

# -------------------------------
# ⚙️ File Configuration
# -------------------------------
csv_file = 'C:/Users/ary33/Desktop/Proj_data_analysis/Vendor/data/sales.csv'  # 👉 change this path
table_name = "sales"
chunksize = 500000  # adjust as per memory

# -------------------------------
# 🧠 Step 1: Read Sample to Detect Columns
# -------------------------------
sample = pd.read_csv(csv_file, nrows=1000)
print("📋 Columns detected:", list(sample.columns))

# Map pandas dtypes to MySQL-compatible types
def map_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return Integer()
    elif pd.api.types.is_float_dtype(dtype):
        return Float()
    else:
        return String(255)

metadata = MetaData()
columns = [Column(col, map_dtype(sample[col].dtype)) for col in sample.columns]
table = Table(table_name, metadata, *columns)

# -------------------------------
# 🧱 Step 2: Create Table in MySQL
# -------------------------------
metadata.drop_all(engine, [table], checkfirst=True)  # drop if exists (optional)
metadata.create_all(engine)
print(f"✅ Table `{table_name}` created successfully in database `{database}`")

# -------------------------------
# 🚀 Step 3: Insert Data in Chunks
# -------------------------------
start_time = time.time()
total_inserted = 0

try:
    for i, chunk in enumerate(tqdm(pd.read_csv(csv_file, chunksize=chunksize))):
        chunk.to_sql(table_name, con=engine, if_exists='append', index=False)
        total_inserted += len(chunk)
        print(f"✅ Chunk {i+1} inserted ({len(chunk)} rows)")

    print(f"\n🎉 Upload complete! Total rows inserted: {total_inserted:,}")
    print(f"⏱️ Time taken: {round(time.time() - start_time, 2)} seconds")

except Exception as e:
    print("❌ Error occurred:", e)


📋 Columns detected: ['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity', 'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification', 'ExciseTax', 'VendorNo', 'VendorName']
✅ Table `sales` created successfully in database `vendor`


1it [00:24, 24.37s/it]

✅ Chunk 1 inserted (500000 rows)


2it [00:48, 24.28s/it]

✅ Chunk 2 inserted (500000 rows)


...

✅ Chunk 24 inserted (500000 rows)


25it [09:54, 23.90s/it]

✅ Chunk 25 inserted (500000 rows)


26it [10:11, 23.53s/it]

✅ Chunk 26 inserted (325363 rows)

🎉 Upload complete! Total rows inserted: 12,825,363
⏱️ Time taken: 611.75 seconds
